### Solar Panel Placement Optimization Notebook 

We recommend to write your code in Python, so that your workflow can be seen using this notebook. However, feel free to use other languages and tools that you are comfortable with. 

Please generate images to illustrate how your algorithm works. 

Have fun! 

Run this cell first to define variables and import neccessary libraries

In [118]:
from shapely import Polygon, buffer, get_coordinates, bounds, covers, prepare, affinity
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as mplPolygon

#* All numbers are in meters
testCoordinate1 = [(0, 0), (0, 100), (50, 0)]
testCoordinate2 = [(1.12, 130.83), (1.12, 160.83), (31.12, 160.83), (31.12, 130.83), (11.12, 100.83), (-11.12, 90.83), (-30.12, 90.83), (-20.182, 120.273)]
testCoordinate3 = [(2, 100), (2,160), (131, 160), (35, 200), (11, 300), (-20 , 90), (30, 90)]

panelSizes = [(2,1),(2.3,1),(1.8,0.9)] #* (width,heigh)
panelGap = 0.02
rowSpacing = 0.5

COORDS = testCoordinate3
PRECISION = 5

I'll start by getting the areas of the different rooftops - this will give us a benchmark of our layout's coverage. 
Formula for calculating area of shape using coordinates of vertices:

|((x1y2 - y1x2) + (x2y3 - y2x3) ... + (xny1 - ynx1)) / 2 |

In [119]:

def getRoofArea(polygon):
  # numerator = 0
  # for i in range(len(coords)):
  #   nextIndex = (i + 1) % len(coords)
  #   numerator += (coords[i][0] * coords[nextIndex][1]) - (coords[i][1] * coords[nextIndex][0])
  # area = abs(numerator / 2)
  # return area
  
  #* Scratch that, I found a library (shapely) that deals with geometric objects
  return polygon.area

# Helper Functions -

In [120]:
#* returns an array [half of width, half of length] of given panel size.
def getOffsetDistance(panelSize):
  return [panelSize[0]/2, panelSize[1]/2]

#* returns the area of 1 panel
def getPanelArea(panelSize):
  return panelSize[0] * panelSize[1]

def getCoverageArea(grid, panelSize):
  return len(grid) * getPanelArea(panelSize)

Starting Point - a point within the roof geometry that a solar panel is guaranteed to fit inside. 

The function will offset the polygon inwards by half of the longest side of the chosen solar panel, and return the point on this offset curve with the smallest value of X and Y (bottom left)

In [121]:
#* not used

def getStartingPoint(polygon, panelSize):
  offsetDistance = max(getOffsetDistance(panelSize)) * -1
  offsetPolygon = buffer(polygon, offsetDistance, cap_style="square", join_style="mitre")
  minX, minY, maxX, maxY = bounds(offsetPolygon)
  offsetCoordinates = get_coordinates(offsetPolygon).tolist()
  startingPoint = []
  
  for offsetCoord in offsetCoordinates:
    if offsetCoord[1] == minY:
      if not startingPoint:
        startingPoint = offsetCoord
      elif offsetCoord[0] < startingPoint[0]:
          startingPoint = offsetCoord
  
  return offsetCoordinates,startingPoint


# offset, startingPoint = getStartingPoint(testCoordinate3, panelSizes[0])
# print(startingPoint)

Alternatively, I could directly offset the minX, minY point and start my grid from there - this will be the method I implemented. 

Using the starting point and the offset bounding box as the bounds, I generate a grid of points which will be the <u>center points</u> of each individual solar panel.

This grid is merely an entire grid of points within the bounding box. I will use an additional function below to filter out the valid points where solar panels can be placed.

In [122]:
def getGrid(polygon, panelSize, shiftX = 0, shiftY = 0):
  offsetX, offsetY = getOffsetDistance(panelSize)
  
  minX, minY, maxX, maxY = bounds(polygon)

  def meterToMM(num):
    return int(1000 * num)
  
  def mmToMeter(num):
    return num/1000

  #* redefine all variables in mm as to avoid errors in using float values
  startXmm = meterToMM(minX + offsetX)
  startYmm = meterToMM(minY + offsetY)
  maxXmm = meterToMM(maxX - offsetX)
  maxYmm = meterToMM(maxY - offsetY)
  shiftXmm = meterToMM(shiftX)
  shiftYmm = meterToMM(shiftY)
  rowSpacingmm = meterToMM(rowSpacing)
  panelGapmm = meterToMM(panelGap)
  panelXmm = meterToMM(panelSize[0])
  panelYmm = meterToMM(panelSize[1])
  
  grid = []
  
  for y in range(startYmm + shiftYmm, maxYmm, panelYmm + rowSpacingmm):
    for x in range(startXmm + shiftXmm, maxXmm, panelXmm + panelGapmm):
      #* conversion back to Meters
      grid.append([mmToMeter(x) ,mmToMeter(y)])
    
  return grid

This function will take each point in the grid and extrapolate the 4 corner points of the solar panel, then check if all corners are "contained" within the roof polygon. If yes, then this is a valid solar panel placement.

In [123]:
def panelsOnRoof(polygon, grid, panelSize):
  validGrid = []
  offsetX, offsetY = getOffsetDistance(panelSize)
  for point in grid:
    x,y = point
    minusX = x - offsetX
    addX = x + offsetX
    minusY = y - offsetY
    addY = y + offsetY
    
    topLeft = (minusX, addY)
    topRight = (addX, addY)
    bottomLeft = (minusX, minusY)
    bottomRight = (addX, minusY)
    
    panelCoords = [topLeft, topRight,bottomRight,bottomLeft]
    panelPolygon = Polygon(panelCoords)
    
    if covers(polygon,panelPolygon):
      validGrid.append(panelCoords)
    
  return validGrid 

Create a function to visualise the rooftop shapes using matplotlib

In [124]:
def visualiseRooftop(roofCoords, panelGrid):
  x = []
  y = []
  fig, ax = plt.subplots()

  for coord in roofCoords:
    # plt.plot(coord[0], coord[1], marker = "o")
    x.append(coord[0])
    y.append(coord[1])
    
  x.append(roofCoords[0][0])
  y.append(roofCoords[0][1])

  ax.plot(x,y, color="black")
  ax.fill(x,y, color="lightgrey")

  for panelCorners in panelGrid:
    panelPolygon = mplPolygon(panelCorners, closed=True, facecolor="mediumblue")
    ax.add_patch(panelPolygon)
    # plt.plot(panel[0],panel[1], marker = "s")
    
  ax.autoscale()
  ax.set_aspect("equal", "box")
  plt.show()
  
  return

## Algorithm discussion
During my [M.Arch thesis](https://www.ar-dan.com/what-the-farm), I used Rhino and Grasshopper to run simulations on building rooftops and facades - the general idea was to see if any slight rotations in buildings could lead to significantly more sunlight for vertical farming. The program used an evolutionary algorithm to return the ideal configuration.

Doing something similar was my first thought when I saw the assignment, but thinking deeper about it, I don't think its the correct path to go down. The key differences was that the daylight simulations I had previously used were very computationally intensive, taking 3-5min per simulation, and I had about 5 buildings to rotate. Using an evolutionary algorithm meant that I didn't need to test all possible permutations, but could narrow down quickly and reduce computation time. Perhaps this is something that could be explored further in the future.

Additionally, since one of the assignment's objective is to assess the quality of my thought process, I will work to develop my algorithm rather than just plugging the data into an already developed algorithm.

My first instinct was that I should start without rotation, and just differing panel sizes. However, after more time spent thinking about the problem, my hypothesis (and what I will build my algorithm off) is that:
- The **rotation will actually have the largest impact on coverage** (may not apply to all cases, will discuss my assumptions later). 
- Choose the best rotation (perhaps include more rotation values if the preliminary results are close)
- Test all combinations of panel sizes and shifting (x & y)

### Rotation
There are 2 main optimisations that I think will significantly reduce the computational requirements.
- Instead of rotating the solar panels itself (which will make keeping the aligned rows and columns difficult), we can simply <u>rotate the roof</u> to achieve the same result. Thus, the solar panels will always be aligned orthogonally. 
- Instead of testing rotation based on a fixed "step" (e.g testing in increments of 5 degrees), my instinct tells me that the best orientations would be when at least <u>1 edge is at 0 degrees</u>. 

Using trigonometry, I can calculate the angle of the edge with the coordinates of adjacent vertices. It is important to include the normal (90 degree) rotation as well, to account for the 2 possible orientations of the solar panel itself.



In [125]:
import math

def getEdgeAngles(roofCoords):
  angles = set()
  for i in range(len(roofCoords)):
    currPoint = roofCoords[i]
    nextPoint = roofCoords[(i + 1) % len(roofCoords)]
    
    #* calculate the angle using arc tangent (opposite/adjacent)
    height = nextPoint[1] - currPoint[1]
    length = nextPoint[0] - currPoint[0]
    angle = math.degrees(math.atan2(height,length))
    
    #* check if opposite of angle is already in the set (no point checking both 30 degrees and 210 degrees (180 + 30))
    if angle >= 180:
      if angle - 180 not in angles:
        angles.add(angle)
        angles.add((angle + 90) % 180)
    else:
      angles.add(angle)
      angles.add((angle + 90) % 180)
      
  return list(angles)

In [126]:
def main():
  roofPolygon = Polygon(COORDS)
  
  #* get roof angles
  angles = getEdgeAngles(COORDS)
  
  #* a quick test for my hypothesis - to check if "odd" angles can achieve better results. Fortunately my hypothesis seems to be correct.
  #* this can significantly reduce the computational requirements
  # angles = []
  # for num in range(0, 360, 5):
  #   angles.append(num)
  
  maxCoverage = 0
  
  for angle in angles:
    rotatedRoof = affinity.rotate(roofPolygon, angle)
    
    #* preparing will reduce computational load for complex roof shapes
    prepare(rotatedRoof)
  
    gridOfCenters = getGrid(rotatedRoof, panelSizes[0])
    gridOfValidPanels = panelsOnRoof(rotatedRoof,gridOfCenters, panelSizes[0])
    coverageArea = getCoverageArea(gridOfValidPanels, panelSizes[0])
    if coverageArea > maxCoverage:
      maxCoverage = coverageArea
      bestRoof = rotatedRoof
      bestAngle = angle
    print("angle:", angle, "\n coverage area:", coverageArea)
  
  print(bestAngle, maxCoverage)
  bestXY = (0,0)
  bestPanel = panelSizes[0]
  
  for panelSize in panelSizes:
    for shiftXfrac in range(1,PRECISION):
      for shiftYfrac in range(1,PRECISION):
        shiftX = panelSize[0] * (shiftXfrac / PRECISION)
        shiftY = panelSize[1] * (shiftYfrac / PRECISION)
        gridOfCenters = getGrid(bestRoof, panelSize, shiftX, shiftY)
        gridOfValidPanels = panelsOnRoof(bestRoof, gridOfCenters, panelSize)
        coverageArea = getCoverageArea(gridOfValidPanels, panelSize)
        print(coverageArea, panelSize, shiftX, shiftY)
        if coverageArea > maxCoverage:
          maxCoverage = coverageArea
          bestXY = shiftX, shiftY
          bestPanel = panelSize
          bestGrid = gridOfValidPanels
          
  print("BEST:", bestAngle, bestXY, bestPanel, maxCoverage)
  
  # roofArea = getRoofArea(roofPolygon)
  # print("Roof Area:", roofArea)
  
  # visualiseRooftop(COORDS, gridOfValidPanels)
  
main()

angle: 0.0 
 coverage area: 4140
angle: 160.3461759419467 
 coverage area: 4068
angle: 67.38013505195957 
 coverage area: 4094
angle: 70.34617594194671 
 coverage area: 4106
angle: 103.49573328079582 
 coverage area: 4132
angle: 171.60269741331024 
 coverage area: 4100
angle: 13.495733280795832 
 coverage area: 4120
angle: 90.0 
 coverage area: 4188
angle: 157.38013505195957 
 coverage area: 4054
angle: -98.39730258668975 
 coverage area: 4134
90.0 4188
4184 (2, 1) 0.4 0.2
4182 (2, 1) 0.4 0.4
4194 (2, 1) 0.4 0.6
4208 (2, 1) 0.4 0.8
4054 (2, 1) 0.8 0.2
4068 (2, 1) 0.8 0.4
4066 (2, 1) 0.8 0.6
4084 (2, 1) 0.8 0.8
4094 (2, 1) 1.2 0.2
4116 (2, 1) 1.2 0.4
4118 (2, 1) 1.2 0.6
4118 (2, 1) 1.2 0.8
4140 (2, 1) 1.6 0.2
4150 (2, 1) 1.6 0.4
4166 (2, 1) 1.6 0.6
4166 (2, 1) 1.6 0.8
4176.799999999999 (2.3, 1) 0.45999999999999996 0.2
4188.299999999999 (2.3, 1) 0.45999999999999996 0.4
4192.9 (2.3, 1) 0.45999999999999996 0.6
4204.4 (2.3, 1) 0.45999999999999996 0.8
4038.7999999999997 (2.3, 1) 0.9199999999

### Limitations:

- Circular roofs, or roofs with many many edges. Roof geometry should ideally be "clean".